In [2]:
import pandas as pd
import altair as alt

In [3]:
df = pd.read_csv('../data/train_data.zip')

In [4]:
df.head()

,external_id,month,year,monthly_number_of_sessions,monthly_unique_sessions,monthly_repeated_sessions,monthly_avg_length_of_session,monthly_avg_light_activity,monthly_avg_moderate_activity,monthly_avg_vigorous_activity,...,avg_wind_9_10,avg_wind_10_11,avg_wind_11_12,avg_wind_12_above,perfect_days,unacast_session_count,hpi,state_and_local_amount_per_capita,state_amount_per_capita,local_amount_per_capita
0,1900203,3,2019,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,78.0,323.61,0.132207,0.018519,0.113688
1,1900203,6,2018,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,111.0,323.61,0.132207,0.018519,0.113688
2,1900203,8,2018,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,110.0,323.61,0.132207,0.018519,0.113688
3,MR00101775,1,2019,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10.0,110.38,0.076247,0.011966,0.064281
4,MR00101775,8,2019,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,11.0,110.38,0.076247,0.011966,0.064281


In [5]:
# Calculate percentage of missing values
missing = df.isna()
num_missing = missing.sum().sort_values(ascending=False)
prop_missing = num_missing / df.shape[0]
df_prop_missing = prop_missing.to_frame().reset_index()
df_prop_missing = df_prop_missing.rename(columns={'index':'col_name', 0:'prop'})

In [6]:
df_20 = df_prop_missing.query('prop < 0.2')
df_20.shape[0]

662

In [7]:
df_20_40 = df_prop_missing.query('0.2 <= prop < 0.4')
df_20_40.shape[0]

19

In [8]:
df_40_60 = df_prop_missing.query('0.4 <= prop < 0.6')
df_40_60.shape[0]

31

In [9]:
df_60_80 = df_prop_missing.query('0.6 <= prop < 0.8')
df_60_80.shape[0]

37

In [10]:
df_80_100 = df_prop_missing.query('0.8 <= prop')
df_80_100.shape[0]

112

In [13]:
nan_rate = (df.isna().sum(axis=0)/df.shape[0]).to_frame().rename(columns={0: "Nan_rate"})
vals = [0.2,0.4,0.6,0.8,1]
count_list = [(nan_rate < 0.2).sum()[0]]
for i in range(len(vals)-1):
    a = ((nan_rate > vals[i]) * (nan_rate <= vals[i+1])).sum()
    count_list.append(a[0])
    
count_list

[662, 19, 31, 37, 112]

In [15]:
# Check that counts add up to 861
print(count_list, sum(count_list))

[662, 19, 31, 37, 112] 861


In [16]:
# Plot histogram binned by 10%
alt.Chart(nan_rate).mark_bar().encode(
    alt.X("Nan_rate:Q", bin=alt.Bin(maxbins=10),title="% NaN in column"),
    y=alt.Y('count()', title="Count")).properties(
    title='Number of columns by percentage of missing values'
)

alt.Chart(...)

In [17]:
# Plot histogram binned by 20%
alt.Chart(nan_rate).mark_bar().encode(
    alt.X("Nan_rate:Q", bin=alt.Bin(maxbins=5),title="% NaN in column"),
    y=alt.Y('count()', title="Count")).properties(
    title='Number of columns by percentage of missing values'
)

alt.Chart(...)